In [1]:
#niezbędne biblioteki
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense, Conv2D, Flatten
from keras.optimizers import Adam
from keras.utils import np_utils
from skimage import color

Using TensorFlow backend.


In [49]:
env = gym.make('Enduro-v0')
env.reset()
goal_steps = 4800
#score_requirement = 6000
intial_games = 10

In [4]:
#do zobaczenia informacji jakie zwraca nam gra
def play_a_random_game_first():
    for step_index in range(goal_steps):
        #env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
        if done:
            break
    env.reset()

In [3]:
#funkcja do preprocesingu ramek. Ucinam górną część (niesitotne niebo) i zmieniam na czarno-biały obrazek
def preprocces_frame(frame):
    img = color.rgb2gray(frame)
    img=img[51:,:]
    img=img.reshape(img.shape[0], img.shape[1],-1)
    return img

In [31]:
#generowanie danych. Zapisuje tylko ścieżki, które dały dobry rezultat
def model_data_preparation(require_score):
    training_data = []
    observations=[]
    actions=[]
    i=0
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 8)
            observation, reward, done, info = env.step(action)
            observation = preprocces_frame(observation)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += 1
            if done:
                break
        if score >= require_score:
            for data in game_memory:
                data[1]=np_utils.to_categorical(data[1],8).reshape(8)
                observations.append(data[0])
                actions.append(data[1])

        env.reset()
        print(i)
        i+=1
    
    return [observations, actions]


In [50]:
training_data = model_data_preparation(4460)

0
1
2
3
4
5
6
7
8
9


In [51]:
training_data

[[array([[[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.19077333],
          [0.19077333],
          [0.19077333]],
  
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.19077333],
          [0.19077333],
          [0.19077333]],
  
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.19077333],
          [0.19077333],
          [0.19077333]],
  
         ...,
  
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]],
  
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]],
  
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]]]), array(

In [17]:
#Funkcja do budowania modelu: prawdopodobnie będę generował i uczył model jednocześnie, by oszczędzić pamięć
def build_model():
    model = Sequential()
    model.add(Conv2D(30, (3,3), input_shape=(159,160, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam())

    return model


In [18]:
def train_model(training_data):
    X = np.array(training_data[0])
    y = np.array(training_data[1])
    model = build_model()
    
    model.fit(X, y, epochs=10)
    return model

In [52]:
trained_model = train_model(training_data)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[744180,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_1/Adam/mul_11}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_1/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [29]:
#Testowanie
scores = []
#choices = []
for each_game in range(20):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, prev_obs.shape[0], prev_obs.shape[1],
                                                                      prev_obs.shape[2])))
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = preprocces_frame(new_observation)
        score+=1
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))
#print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

[4443, 4431, 4456, 4436, 4411, 4444, 4442, 4448, 4430, 4438, 4435, 4440, 4461, 4435, 4391, 4440, 4429, 4412, 4450, 4429]
Average Score: 4435.05


In [30]:
#Dla porównania losowe ruchy
scores = []
#choices = []
for each_game in range(20):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        action = random.randrange(0,8)
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=1
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))

[4432, 4387, 4467, 4450, 4463, 4422, 4417, 4462, 4418, 4414, 4466, 4414, 4437, 4444, 4409, 4441, 4443, 4473, 4454, 4448]
Average Score: 4438.05
